In [1]:
# Importing the csv module to work with csv files
import csv 

# Importing intervaltree and its Interval, IntervalTree classes for efficient interval operations
import intervaltree
from intervaltree import Interval, IntervalTree 

# Importing pandas for data manipulation and analysis
import pandas as pd

# Importing copy for creating copies of mutable objects
import copy

# Importing numpy for numerical operations
import numpy as np

# Importing matplotlib's pyplot for data visualization
import matplotlib.pyplot as plt

In [2]:
#!pip install intervaltree

In [3]:
def find_smallest_region(genome):
    # Initialize smallest with a very large number
    smallest=1000000000000000000

    # Iterate over each item in the genome
    for i in genome:
        # Iterate over the list [-1, 0, 1]
        for j in [-1,0,1]:
            # Calculate the absolute difference between the end of the current interval and the beginning of the next interval
            x=np.abs(list(genome[i])[j].end-list(genome[i])[j+1].begin)

            # If the calculated difference is smaller than the current smallest, update smallest
            if x<smallest:
                smallest=x
                # Print the index of the smallest region
                print(i)
    # Return the smallest region
    return smallest

In [4]:
# Define the file name
file_name="files/telocentro_hg38.bed"

# Open the file in read mode
file=open(file_name,"r")

# Read all the lines from the file and store them in the data variable
data=file.readlines()

In [5]:
# Initialize an empty dictionary to store the genome data
genome={}

# Iterate over each line in the data
for i in data:
    # Split the line by tab character to get the individual fields
    x=i.split("\t")

    # If the first field (x[0]) is not already a key in the genome dictionary
    if x[0] not in genome.keys():
        # Create a new IntervalTree for this key
        genome[x[0]]=IntervalTree()

        # Add a new interval to the IntervalTree with the second and third fields as the start and end of the interval
        # The value of the interval is a tuple of the start and end
        genome[x[0]][int(x[1]):int(x[2])]=(int(x[1]),int(x[2]))

    # If the first field (x[0]) is already a key in the genome dictionary
    else:
        # Add a new interval to the existing IntervalTree with the second and third fields as the start and end of the interval
        # The value of the interval is a tuple of the start and end
        genome[x[0]][int(x[1]):int(x[2])]=(int(x[1]),int(x[2]))
        
# Iterate over each key in the genome dictionary
for key in genome.keys():
    # Merge overlapping intervals in the IntervalTree for this key
    # The strict parameter is set to False, which means that adjacent intervals will be merged
    genome[key].merge_overlaps(strict=False)

# Delete the key "chrX" from the genome dictionary
del genome["chrX"]

# Delete the key "chrY" from the genome dictionary
del genome["chrY"]

In [6]:
find_smallest_region(genome)

chr1
chr2
chr3
chr4
chr5
chr8
chr10
chr12
chr13
chr14
chr18
chr21


10890000

In [7]:
genome

{'chr1': IntervalTree([Interval(0, 10000, (0, 10000)), Interval(121700000, 125100000), Interval(248946422, 248956422, (248946422, 248956422))]),
 'chr2': IntervalTree([Interval(0, 10000, (0, 10000)), Interval(91800000, 96000000), Interval(242183529, 242193529, (242183529, 242193529))]),
 'chr3': IntervalTree([Interval(0, 10000, (0, 10000)), Interval(87800000, 94000000), Interval(198285559, 198295559, (198285559, 198295559))]),
 'chr4': IntervalTree([Interval(0, 10000, (0, 10000)), Interval(48200000, 51800000), Interval(190204555, 190214555, (190204555, 190214555))]),
 'chr5': IntervalTree([Interval(0, 10000, (0, 10000)), Interval(46100000, 51400000), Interval(181528259, 181538259, (181528259, 181538259))]),
 'chr6': IntervalTree([Interval(0, 10000, (0, 10000)), Interval(58500000, 62600000), Interval(170795979, 170805979, (170795979, 170805979))]),
 'chr7': IntervalTree([Interval(0, 10000, (0, 10000)), Interval(58100000, 62100000), Interval(159335973, 159345973, (159335973, 159345973))]

In [8]:
# Function to split the genome into smaller regions
def get_splits(chromosome_name):
    # Initialize an IntervalTree to store the splits
    splits=IntervalTree()

    # Get the sorted list of intervals for the given chromosome
    x=sorted(genome[chromosome_name])
    print(len(x))

    # Define the length of each split
    split_length=(10**7)

    # Iterate over each interval in the list (except the last one)
    for i in range(len(x)-1):
        # Get the end of the current interval and the beginning of the next interval
        s=x[i].end
        e=x[i+1].begin

        # Create splits from the end of the current interval to the middle of the gap between the current and next interval
        for i in range(s+1,(s+e)//2,split_length+1):
            splits[i:i+split_length]=(i,i+split_length)

        # Create splits from the beginning of the next interval to the middle of the gap between the current and next interval
        for i in range( e-1 , (s+e)//2 + (e-s)%(split_length+1) , -split_length-1): #  -split_length-1 
            splits[i-split_length:i]=(i-split_length,i)

    # Return the IntervalTree of splits
    return splits

In [9]:
splits=get_splits("chr21")

3


In [10]:
len(splits)

6

In [11]:
len(splits)

6

In [12]:
duplicated_regions=pd.read_csv("files/out_df_ws_jumps.csv")

In [13]:
duplicated_regions

,chr,coor_s,coor_e,ids,jumps,length,centro,telo,gaps,genes,...,MIR_s_r,Alu_s_r,Satellite_s_r,used_coor_l_s,used_coor_l_e,used_coor_r_s,used_coor_r_e,CG_frac_l,CG_frac_r,CG_frac_in
0,1,10000,207666,id1,3,197666,121818793,0,2,6,...,0,0,0,9950,10000,207666,207716,-1.000,0.000,0.444
1,1,257666,297956,id2,3,40290,121728503,247666,2,0,...,0,1,0,257616,257666,297956,298006,-1.000,0.538,0.391
2,1,347968,535988,id3,1,188020,121490471,337968,2,1,...,0,0,0,347918,347968,535988,536038,-1.000,1.000,0.430
3,1,585988,817292,id4,4,231304,121209167,575988,1,2,...,0,0,0,585938,585988,817292,817342,-1.000,0.412,0.428
4,1,817367,821400,id5,1,4033,121205059,807367,0,1,...,0,0,0,817317,817367,821400,821450,0.392,0.529,0.510
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6651,22,48911634,48912886,id4359,1,1252,33857316,1895582,0,0,...,0,0,0,48911584,48911634,48912886,48912936,0.490,0.451,0.468
6652,22,49383944,49385910,id4360,1,1966,34329626,1422558,0,0,...,0,0,0,49383894,49383944,49385910,49385960,0.510,0.392,0.393
6653,22,49386637,49388496,id4361,1,1859,34332319,1419972,0,0,...,0,0,0,49386587,49386637,49388496,49388546,0.392,0.314,0.397
6654,22,50432257,50442552,id4362,2,10295,35377939,365916,0,1,...,0,1,0,50432207,50432257,50442552,50442602,0.725,0.451,0.648


In [14]:
def get_duplicated_regions():
    # Define the columns to be added from the old data
    add_columns=['component_size', 'intra_degree', 'iner_degree', 'self_loops', 'edges_double', 'edges_tandem','edges_ident_mean']

    # Read the old data and the duplicated regions data from CSV files
    old_data=pd.read_csv("files/Duplicated_Regions_old_data.csv")
    duplicated_regions=pd.read_csv("files/out_df_ws_jumps.csv")

    # Replace -1.000 with 0.0 in the CG_frac_l, CG_frac_r, and CG_frac_in columns
    # Replace 0.0 with the corresponding value from the CG_frac_r or CG_frac_l column or the mean of the CG_frac_in column
    duplicated_regions["CG_frac_l"][duplicated_regions["CG_frac_l"]==-1.000]=0.0
    duplicated_regions["CG_frac_l"][duplicated_regions["CG_frac_l"]==0.0]=duplicated_regions["CG_frac_r"][duplicated_regions["CG_frac_l"]==0.0]
    duplicated_regions["CG_frac_r"][duplicated_regions["CG_frac_r"]==-1.000]=0.0
    duplicated_regions["CG_frac_r"][duplicated_regions["CG_frac_r"]==0.0]=duplicated_regions["CG_frac_l"][duplicated_regions["CG_frac_r"]==0.0]
    duplicated_regions["CG_frac_in"][duplicated_regions["CG_frac_in"]==-1.000]=0.0
    duplicated_regions["CG_frac_in"][duplicated_regions["CG_frac_in"]==0.0]=np.mean(duplicated_regions["CG_frac_in"])

    # Calculate the mean of the CG_frac_l and CG_frac_r columns and store it in the CG_frac column
    duplicated_regions["CG_frac"]=(duplicated_regions["CG_frac_l"]+duplicated_regions["CG_frac_r"])/2

    # Replace 0.0 in the CG_frac column with the mean of the CG_frac_in column
    duplicated_regions["CG_frac"][duplicated_regions["CG_frac"]==0.0]=np.mean(duplicated_regions["CG_frac_in"])

    # Drop the CG_frac_l and CG_frac_r columns
    duplicated_regions.drop(["CG_frac_l","CG_frac_r"],axis=1,inplace=True)

    # Add the columns from the old data to the duplicated regions data
    for i in add_columns:
        duplicated_regions[i]=old_data[i]

    # Define the columns that have a left and right version
    left_right_columns=["DNA","LINE","LTR","SINE","Low_complexity","Retroposon","Satellite","Simple_repeat","rRNA","snRNA","scRNA","srpRNA","tRNA","RC", 'L1_s', 'L2_s', 'MIR_s', 'Alu_s', 'Satellite_s']

    # Combine the left and right versions of the columns and drop the original columns
    for i in left_right_columns:
        duplicated_regions[i]=duplicated_regions[i+"_r"]+duplicated_regions[i+"_l"]
        duplicated_regions.drop([i+"_r",i+"_l"],axis=1,inplace=True)

    # Return the processed duplicated regions data
    return duplicated_regions

In [15]:
def find_duplicated_regions(splits, duplicated_regions, chromosome_name):
    # Create a deep copy of the splits IntervalTree
    splits2 = copy.deepcopy(splits)

    # Filter the duplicated regions DataFrame for the given chromosome
    df = duplicated_regions[duplicated_regions["chr"] == chromosome_name]

    # For each row in the filtered DataFrame, add an interval to the splits2 IntervalTree
    # The interval begins at the start coordinate and ends at the end coordinate of the row
    # The data of the interval is the ID of the row
    for i, row in df.iterrows():
        splits2[row["coor_s"]:row["coor_e"]] = row["ids"]

    # Initialize a dictionary to store the intervals in the splits IntervalTree that overlap with the intervals in the splits2 IntervalTree
    interval_with_duplicated_regions = {}

    # For each interval in the splits IntervalTree, find the overlapping intervals in the splits2 IntervalTree
    # Remove the interval itself from the list of overlaps
    # Add the list of overlaps to the dictionary with the interval as the key
    for i in splits:
        overlaps = splits2.overlap(i.begin, i.end)
        overlaps.remove(Interval(i.begin, i.end, (i.begin, i.end)))
        interval_with_duplicated_regions[(i.begin, i.end)] = overlaps

    # Return the dictionary of intervals with overlapping intervals
    return interval_with_duplicated_regions

In [16]:
def length_of_overlap(reg1,reg2):
    if reg1[0]<=reg2[0]:
        return(reg1[1]-reg2[0])
    else:
        return(reg2[1]-reg1[0])

In [17]:
def binning(x):
    if x>0.933:
        return 1
    else:
        return 0

In [18]:
def binning1(x): #(1.745, 0.196)
    if x>0.40174999999999994:
        return 1
    else:
        return 0

In [19]:
def binning2(x): #0.749, 0.065
    if x>0.424:
        return 1
    else:
        return 0

In [20]:
def binning_length(x):
    if x<2574.5:
        return 0 
    else:
        return 1

In [21]:
columns_needed=["chromosome_name","start", "end",'length_0','length_1', 'jumps',  'gaps', 'genes',
       'cpgisl_in', 'cpgisl_bor', 'repli_in', 'repli_bor', 'repli_bor_deriv',
       'repli_deriv', 'recomb_in', 'recomb_bor', 'dnase_in', 'dnase_bor','DNA', 'LINE',
       'LTR', 'SINE', 'Low_complexity', 'Retroposon', 'Satellite',
       'Simple_repeat', 'rRNA', 'snRNA', 'scRNA', 'srpRNA', 'tRNA', 'RC',
       'L1_s', 'L2_s', 'MIR_s', 'Alu_s', 'Satellite_s','component_size', 'intra_degree', 'iner_degree', 'self_loops',
       'edges_double', 'edges_tandem','edges_ident_mean_0','edges_ident_mean_1',"CG_frac_0","CG_frac_1","CG_frac_in_0","CG_frac_in_1"]

In [22]:
len(columns_needed)

49

In [23]:
def create_training_data(duplicated_regions):
    # Define the columns needed for the DataFrame
    columns_needed = ["chromosome_name", "start", "end", 'length_0', 'length_1', 'jumps', 'gaps', 'genes',
                      'cpgisl_in', 'cpgisl_bor', 'repli_in', 'repli_bor', 'repli_bor_deriv',
                      'repli_deriv', 'recomb_in', 'recomb_bor', 'dnase_in', 'dnase_bor', 'DNA', 'LINE',
                      'LTR', 'SINE', 'Low_complexity', 'Retroposon', 'Satellite',
                      'Simple_repeat', 'rRNA', 'snRNA', 'scRNA', 'srpRNA', 'tRNA', 'RC',
                      'L1_s', 'L2_s', 'MIR_s', 'Alu_s', 'Satellite_s', 'component_size', 'intra_degree',
                      'iner_degree', 'self_loops', 'edges_double', 'edges_tandem', 'edges_ident_mean_0',
                      'edges_ident_mean_1', "CG_frac_0", "CG_frac_1", "CG_frac_in_0", "CG_frac_in_1"]

    df = pd.DataFrame(columns=columns_needed)  # Initialize an empty DataFrame to store the training data

    for i in range(1, 23):  # Iterate over chromosome numbers (1 to 22)
        print("Processing Chromosome:", i)
        chr_name = "chr" + str(i)  # Construct the chromosome name
        splits = get_splits(chr_name)  # Get the split points for the chromosome
        overlap_data = find_duplicated_regions(splits, duplicated_regions, i)  # Find duplicated regions for the chromosome

        for split in overlap_data:  # Iterate over each split
            start = split[0]  # Start position of the split
            end = split[1]  # End position of the split
            cumm_features = np.zeros(len(columns_needed))  # Initialize an array for cumulative features
            # Store the chromosome name, start, and end positions of the split in the cumulative features array
            cumm_features[0] = i
            cumm_features[1] = start
            cumm_features[2] = end

            for dups in overlap_data[split]:  # Iterate over duplicated regions within the split

                # Extract relevant row from feature data (duplication region).
                dups_id = dups[2]
                dup_data = features[features["ids"] == dups_id]

                # Extracting overlap length ratio
                if len(dup_data) != 0:
                    length_of_dup = np.array(dup_data["length"])[0]
                    cumm_features[3 + binning_length(length_of_dup)] += 1
                    # Add features related to replication, recombination, and DNase
                    index_repli = 5
                    for count in np.array(
                            dup_data[['jumps', 'gaps', 'genes', 'cpgisl_in', 'cpgisl_bor', 'repli_in',
                                       'repli_bor', 'repli_bor_deriv', 'repli_deriv', 'recomb_in',
                                       'recomb_bor', 'dnase_in', 'dnase_bor', 'DNA', 'LINE', 'LTR',
                                       'SINE', 'Low_complexity', 'Retroposon', 'Satellite', 'Simple_repeat',
                                       'rRNA', 'snRNA', 'scRNA', 'srpRNA', 'tRNA', 'RC', 'L1_s', 'L2_s',
                                       'MIR_s', 'Alu_s', 'Satellite_s', 'component_size', 'intra_degree',
                                       'iner_degree', 'self_loops', 'edges_double', 'edges_tandem',
                                       "CG_frac", "CG_frac_in"]])[0]:
                        cumm_features[index_repli] += int(count)
                        index_repli += 1
                    # Add features related to edges identification mean
                    index_edges_ident_mean = 43
                    cumm_features[
                        index_edges_ident_mean + int(binning(np.array(dup_data['edges_ident_mean'])[0]))] += 1

                    # Add features related to CG fraction
                    index_cg_frac = 45
                    cumm_features[index_cg_frac + int(binning1(np.array(dup_data['CG_frac'])[0]))] += 1

                    # Add features related to CG fraction in
                    index_cg_frac_in = 47
                    cumm_features[index_cg_frac_in + int(binning2(np.array(dup_data['CG_frac_in'])[0]))] += 1

            # Add the row data into DataFrame
            df2 = pd.DataFrame([cumm_features], columns=columns_needed)
            df = pd.concat([df, df2])  # Concatenate the DataFrame with the new row

    return df  # Return the DataFrame containing the training data


In [24]:
features=get_duplicated_regions()

/tmp/ipykernel_34207/3842241318.py:11: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  duplicated_regions["CG_frac_l"][duplicated_regions["CG_frac_l"]==-1.000]=0.0
/tmp/ipykernel_34207/3842241318.py:11: SettingWithCopyWarning: 
A value is tryi

In [25]:
features.to_csv("files/Duplicated_Regions_6.6k.csv",index=False)

In [26]:
df_ML_output=create_training_data(features)

Processing Chromosome: 1
3


/tmp/ipykernel_34207/1885382976.py:67: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, df2])  # Concatenate the DataFrame with the new row


Processing Chromosome: 2
3
Processing Chromosome: 3
3
Processing Chromosome: 4
3
Processing Chromosome: 5
3
Processing Chromosome: 6
3
Processing Chromosome: 7
3
Processing Chromosome: 8
3
Processing Chromosome: 9
3
Processing Chromosome: 10
3
Processing Chromosome: 11
3
Processing Chromosome: 12
3
Processing Chromosome: 13
3
Processing Chromosome: 14
3
Processing Chromosome: 15
3
Processing Chromosome: 16
3
Processing Chromosome: 17
3
Processing Chromosome: 18
3
Processing Chromosome: 19
3
Processing Chromosome: 20
3
Processing Chromosome: 21
3
Processing Chromosome: 22
3


In [27]:
df_ML_output

,chromosome_name,start,end,length_0,length_1,jumps,gaps,genes,cpgisl_in,cpgisl_bor,...,iner_degree,self_loops,edges_double,edges_tandem,edges_ident_mean_0,edges_ident_mean_1,CG_frac_0,CG_frac_1,CG_frac_in_0,CG_frac_in_1
0,1.0,10001.0,10010001.0,7.0,29.0,51.0,9.0,40.0,44.0,2.0,...,140.0,11.0,184.0,28.0,13.0,24.0,5.0,31.0,6.0,30.0
0,1.0,218946419.0,228946419.0,13.0,15.0,36.0,2.0,15.0,44.0,2.0,...,30.0,2.0,96.0,10.0,17.0,11.0,13.0,15.0,13.0,15.0
0,1.0,165100005.0,175100005.0,14.0,6.0,24.0,0.0,15.0,1.0,0.0,...,24.0,0.0,0.0,2.0,12.0,8.0,16.0,4.0,14.0,6.0
0,1.0,125100001.0,135100001.0,0.0,4.0,9.0,4.0,0.0,2.0,1.0,...,62.0,3.0,101.0,2.0,2.0,2.0,1.0,3.0,4.0,0.0
0,1.0,30010004.0,40010004.0,25.0,8.0,35.0,0.0,23.0,7.0,2.0,...,39.0,0.0,0.0,9.0,16.0,17.0,11.0,22.0,8.0,25.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,22.0,10001.0,10010001.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,22.0,17400001.0,27400001.0,23.0,86.0,166.0,6.0,107.0,69.0,4.0,...,77.0,10.0,255.0,80.0,38.0,71.0,25.0,84.0,25.0,84.0
0,22.0,27400002.0,37400002.0,13.0,19.0,39.0,0.0,25.0,3.0,1.0,...,7.0,2.0,2.0,4.0,14.0,18.0,7.0,25.0,11.0,21.0
0,22.0,40808467.0,50808467.0,24.0,17.0,51.0,1.0,34.0,12.0,1.0,...,19.0,0.0,8.0,24.0,19.0,22.0,11.0,30.0,6.0,35.0


In [28]:
df_ML_output.columns

Index(['chromosome_name', 'start', 'end', 'length_0', 'length_1', 'jumps',
       'gaps', 'genes', 'cpgisl_in', 'cpgisl_bor', 'repli_in', 'repli_bor',
       'repli_bor_deriv', 'repli_deriv', 'recomb_in', 'recomb_bor', 'dnase_in',
       'dnase_bor', 'DNA', 'LINE', 'LTR', 'SINE', 'Low_complexity',
       'Retroposon', 'Satellite', 'Simple_repeat', 'rRNA', 'snRNA', 'scRNA',
       'srpRNA', 'tRNA', 'RC', 'L1_s', 'L2_s', 'MIR_s', 'Alu_s', 'Satellite_s',
       'component_size', 'intra_degree', 'iner_degree', 'self_loops',
       'edges_double', 'edges_tandem', 'edges_ident_mean_0',
       'edges_ident_mean_1', 'CG_frac_0', 'CG_frac_1', 'CG_frac_in_0',
       'CG_frac_in_1'],
      dtype='object')

In [29]:
df_ML_output.to_csv("files/Duplicated_Regions_Final_10MB.csv",index=False)